#### "Artist analysis" with the iTunes search API...

In [2]:
import requests

In [3]:
#
# wrap our requests in a function with a specific purpose
#

def get_id_from_name(artist_name):
    """ This function takes in 
           artist_name:  a string, e.g., "Lizzo" or "Taylor Swift" or ...
           and returns the artist's id#

        Return -1 to signal an error if there was no match.
    """
    # Not in the table above? Use the itunes search API to get an artist's itunes ID
    search_url = "https://itunes.apple.com/search"

    parameters = {  "term":artist_name,
                    "entity":"musicArtist",
                    "media":"music",
                    "limit":200  }

    result = requests.get(search_url, params=parameters)
    # print(result.url)
    # print(f"result.status_code is {result.status_code}")
    if result.status_code == 200:
        data = result.json()   # this is _ALL_ the data
        #
        # here, you need to extract the desired artist id!
        id = data['results'][0]['artistId']
        #
        return id
        # for the moment, the above just returns all of the data.
    else:
        print(f"the request result was {result}, returning {{}}")  # wow
        return {}    # return no data 
 

In [4]:
if True:
    artist_names = ["Kendrick Lamar", "Taylor Swift", "Lizzo", "The Beatles"]
    for artist_name in artist_names:
        result = get_id_from_name(artist_name)
        print(f"artist id for {artist_name} is {result}")

artist id for Kendrick Lamar is 368183298
artist id for Taylor Swift is 159260351
artist id for Lizzo is 472949623
artist id for The Beatles is 136975


In [5]:
def get_albums_from_id(id):
    """
    Arguments: an artistId (named id)
    Return: the _whole_ json-obtained dictionary of album data
    """
    #Using the lookup as search_url
    search_url = "https://itunes.apple.com/lookup?"

    #Using the link above as reference, put parameters as "entity" and "id" and "limit"
    parameters = {  "entity":"album",
                    "id":id,
                    "limit":200 }
    
    #Using requests to get search_url with parameters
    result = requests.get(search_url, params=parameters)

    if result.status_code == 200:
        data = result.json()   # this is _ALL_ the data
        #return all the data in json itself
        return data
    else:
        print(f"the request result was {result}, returning {{}}")  # wow
        return {}    # return no data 

get_albums_from_id(64387566)

{'resultCount': 87,
 'results': [{'wrapperType': 'artist',
   'artistType': 'Artist',
   'artistName': 'Katy Perry',
   'artistLinkUrl': 'https://music.apple.com/us/artist/katy-perry/64387566?uo=4',
   'artistId': 64387566,
   'amgArtistId': 1010533,
   'primaryGenreName': 'Pop',
   'primaryGenreId': 14},
  {'wrapperType': 'collection',
   'collectionType': 'Album',
   'artistId': 64387566,
   'collectionId': 1440818542,
   'amgArtistId': 1010533,
   'artistName': 'Katy Perry',
   'collectionName': 'PRISM (Deluxe Version)',
   'collectionCensoredName': 'PRISM (Deluxe Version)',
   'artistViewUrl': 'https://music.apple.com/us/artist/katy-perry/64387566?uo=4',
   'collectionViewUrl': 'https://music.apple.com/us/album/prism-deluxe-version/1440818542?uo=4',
   'artworkUrl60': 'https://is1-ssl.mzstatic.com/image/thumb/Music113/v4/b7/52/f9/b752f9fc-6a70-71d0-87ad-c1b6dcc36a39/source/60x60bb.jpg',
   'artworkUrl100': 'https://is1-ssl.mzstatic.com/image/thumb/Music113/v4/b7/52/f9/b752f9fc-6a70

In [6]:
"""
*** Comparing Artist Productivity ***

Steve Perry: 11 results
Katy Perry: 87 results
(((Katy Perry has more!)))

Adele: 11 results
Lorde: 17 results
(((Lorde has more!)))
"""
def more_productive( artist_name1, artist_name2 ):
    """
    Arguments: two artist names - string
    Print the artist who has produced more albums
    """
    # Getting artist id using get_id_from_name function
    artistId1 = get_id_from_name(artist_name1)
    artistId2 = get_id_from_name(artist_name2)
    # Using the artistID to get album count and then accessing data through the ['resultCount'] key
    albumCount1 = get_albums_from_id(artistId1)['resultCount']
    albumCount2 = get_albums_from_id(artistId2)['resultCount']
    # Formatting print statements
    print(f"{artist_name1}: {albumCount1} results")
    print(f"{artist_name2}: {albumCount2} results")
    if albumCount1 > albumCount2:
        print(f"{artist_name1} has more!")
    elif albumCount2 > albumCount1:
        print(f"{artist_name2} has more!")
    else:
        print(f"{artist_name1} and {artist_name2} have same")
    # Print the space between each run
    print()
    
more_productive("Steve Perry","Katy Perry")
more_productive("Adele","Lorde")
more_productive("Taylor Swift","Katy Perry")

Steve Perry: 12 results
Katy Perry: 87 results
Katy Perry has more!

Adele: 12 results
Lorde: 18 results
Lorde has more!

Taylor Swift: 110 results
Katy Perry: 87 results
Taylor Swift has more!



In [7]:
def albumsMoreThanN(id,n):
    """
    Helper function - return the number of albums that have more than n tracks
    Arguments:
        id - The ID of the artist we want to look up
        num - The minimum number of tracks to look up in an album
    Return: The number of albums that have more than n tracks
    """
    # Storing a counter variable
    count = 0
    # Access data using function get_albums_from_id
    data = get_albums_from_id(id)
    # Parsing through the data using 'results' key
    for i in data['results']:
        # If 'trackCount' is a key and trackCount is more than n then increase count by 1
        if "trackCount" in i and i['trackCount'] > n:
            count += 1
    return count

In [21]:
albumsMoreThanN(64387566, 5)

18

In [8]:
"""
*** Comparing Number of Large Albums ***
"""
def musical_generosity( artist_name1, artist_name2,n):
    """
    Arugments: two artists name - string, threshold of no. albums - integer
    """
    #Getting artist id using get_id_from_name function
    artistId1 = get_id_from_name(artist_name1)
    artistId2 = get_id_from_name(artist_name2)
    #Using the helper function to return albumsMoreThanN where N is also sent into function along with artistID
    numAlbumsN1 = albumsMoreThanN(artistId1,n)
    numAlbumsN2 = albumsMoreThanN(artistId2,n)
    #Formatting Print Statements
    print(f"If we look at albums containing a minimum of {n} tracks...")
    print(f"{artist_name1} : {numAlbumsN1} results")
    print(f"{artist_name2} : {numAlbumsN2} results")
    if numAlbumsN1 > numAlbumsN2:
        print(f"{artist_name1} has more big albums!!")
    elif numAlbumsN2 > numAlbumsN1:
        print(f"{artist_name2} has more big albums!!")
    else:
        print(f"{artist_name1} and {artist_name2} have same number of big albums!!")

musical_generosity("Taylor Swift","Katy Perry",10)

If we look at albums containing a minimum of 10 tracks...
Taylor Swift : 41 results
Katy Perry : 12 results
Taylor Swift has more big albums!!


In [46]:
#most single-word album titles; which has the most album titles with numbers in them
#collectionPrice per track

In [67]:
def numAlbumsWithDigit(artist_name1,artist_name2,artist_name3,artist_name4,artist_name5):
    """
    This function returns the number of albums with digits in them for the 5 artists
    Arguments: 
    artist_name1,artist_name2,artist_name3 - The names of 5 artists
    Output:
    The artist(s) with the most and least albums that have digits in them
    """

    L = [artist_name1,artist_name2,artist_name3,artist_name4,artist_name5]
    List = [[numAlbumsWithDigitHelper(i),i] for i in L]
    maxNumbers = max(List)
    minNumbers = min(List)
    for j in List:
        print(f"{j[1]} : {j[0]} album names containing digits")

    print(f"\n{maxNumbers[1]} had {maxNumbers[0]} albums with numbers in them which is the maximum!")
    print(f"{minNumbers[1]} had {minNumbers[0]} albums with numbers in them which is the minimum!\n\n")

numAlbumsWithDigit("Katy Perry","The Beatles","Rihanna","Jay-Z","J. Cole")
numAlbumsWithDigit("Jay-Z","Kendrick Lamar","Linkin Park","The Beatles","Rihanna")
numAlbumsWithDigit("Olivia Rodrigo","Jason Derulo","Taylor Swift", "Rihanna","Selena Gomez")

Katy Perry : 5 album names containing digits
The Beatles : 17 album names containing digits
Rihanna : 3 album names containing digits
Jay-Z : 15 album names containing digits
J. Cole : 4 album names containing digits

The Beatles had 17 albums with numbers in them which is the maximum!
Rihanna had 3 albums with numbers in them which is the minimum!


Jay-Z : 15 album names containing digits
Kendrick Lamar : 5 album names containing digits
Linkin Park : 5 album names containing digits
The Beatles : 17 album names containing digits
Rihanna : 3 album names containing digits

The Beatles had 17 albums with numbers in them which is the maximum!
Rihanna had 3 albums with numbers in them which is the minimum!


Olivia Rodrigo : 5 album names containing digits
Jason Derulo : 14 album names containing digits
Taylor Swift : 13 album names containing digits
Rihanna : 3 album names containing digits
Selena Gomez : 0 album names containing digits

Jason Derulo had 14 albums with numbers in them whi

In [59]:
def numAlbumsWithDigitHelper(artistName):
    """
    Helper function for singleWordAlbums
    Argument:
    id: The id of an artist
    Return:
    The number of albums with numbers in them
    """
    id = get_id_from_name(artistName)

    #Using get_albums_from_id to extract data
    data = get_albums_from_id(id)
    num_Digit = 0
    # Parsing through the data using 'results' key
    for i in data['results']:
        # If 'trackCount' is a key and trackCount is more than n then increase count by 1
        if "collectionName" in i:
            isThereDigit = [j.isdigit() for j in i["collectionName"]]
            num_Digit += max(isThereDigit) #If there is a single True, num_Digit will be increased by 1
    return num_Digit



In [60]:
def avgPricePerTrack(artistName):
    """
    Helper function for lowest Price per Track
    Arguments:
    artist_name - The name of the artist
    Returns:
    Average Price per track of artist
    """
    id = get_id_from_name(artistName)
    data = get_albums_from_id(id)
    sumPrice,sumTracks = 0,0

    for i in data['results']:
        #We only check if we have data available for both collectionPrice and trackCount
        if 'collectionPrice' in i and 'trackCount' in i:
            sumPrice += i['collectionPrice']
            sumTracks += i['trackCount']
    
    return sumPrice/sumTracks
    

In [68]:
def pricePerTrack(artist_name1,artist_name2,artist_name3,artist_name4,artist_name5):
    """
    This function compares the price per track of 5 artists
    The way we have computed this is collectionPrice/trackCount

    Arguments: 
    artist_name1,artist_name2,artist_name3 = The names of 5 artists

    Output: The Price per track of the 5 artists across all albums
    """
    
    L = [artist_name1,artist_name2,artist_name3,artist_name4,artist_name5]
    List = [[avgPricePerTrack(i),i] for i in L]
    minNumbers = min(List)
    maxNumbers = max(List)

    for j in List:
        print(f"{j[1]} : {j[0]} average price per track across career")

    print(f"\n{minNumbers[1]} had an average price of {minNumbers[0]} over career which is the lowest!")
    print(f"{maxNumbers[1]} had an average price of {maxNumbers[0]} over career which is the highest!\n\n")


pricePerTrack("Katy Perry","The Beatles","Rihanna","Jay-Z","J. Cole")
pricePerTrack("Jay-Z","Kendrick Lamar","Linkin Park","The Beatles","Rihanna")
pricePerTrack("Olivia Rodrigo","Jason Derulo","Taylor Swift", "Rihanna","Selena Gomez")

Katy Perry : 0.8944557823129251 average price per track across career
The Beatles : 0.602138263665595 average price per track across career
Rihanna : 0.8793220338983067 average price per track across career
Jay-Z : 0.7797084548104958 average price per track across career
J. Cole : 0.7357336956521742 average price per track across career

The Beatles had an average price of 0.602138263665595 over career which is the lowest!
Katy Perry had an average price of 0.8944557823129251 over career which is the highest!


Jay-Z : 0.7797084548104958 average price per track across career
Kendrick Lamar : 0.7994242424242419 average price per track across career
Linkin Park : 0.8374135338345865 average price per track across career
The Beatles : 0.602138263665595 average price per track across career
Rihanna : 0.8793220338983067 average price per track across career

The Beatles had an average price of 0.602138263665595 over career which is the lowest!
Rihanna had an average price of 0.87932203389830